In [1]:
import torch
import matplotlib.pyplot as plt

from utils import Dataset, load_data
from trainer import batch_gd, evaluate
from models import DeepLOB

In [2]:
config = {
    # Data configs
    'data_path' : './data/',
    'batch_size' : 32,
    'num_classes' : 3,
    'T' : 100,
    'k' : 10,
    
    # Training configs
    'lr' : 0.01,
    'eps' : 1.0,
    'epochs' : 150,
    'device' : torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'patience': 20,
    'min_delta': 1e-6,
    'print_freq': 10
}
print(config['device'])

cuda


In [3]:
# Load the data
train, val, test = load_data(config['data_path'])

dataset_train = Dataset(data=train, k=config['k'], num_classes=config['num_classes'], T=config['T'])
dataset_val = Dataset(data=val, k=config['k'], num_classes=config['num_classes'], T=config['T'])
dataset_test = Dataset(data=test, k=config['k'], num_classes=config['num_classes'], T=config['T'])

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=config['batch_size'], shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=config['batch_size'], shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=config['batch_size'], shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape)

(149, 203800) (149, 50950) (149, 139587)
torch.Size([203701, 1, 100, 40]) torch.Size([203701])


In [4]:
model = DeepLOB(y_len = dataset_train.num_classes)
model.to(config['device'])

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=config['eps'])      # set as in the paper or use lr=0.0001

In [ ]:
model, train_losses, val_losses, train_accs, val_accs = batch_gd(model,
                                                                 criterion,
                                                                 optimizer,
                                                                 train_loader,
                                                                 val_loader,
                                                                 config
                                                                 )

  1%|          | 1/150 [02:22<5:52:41, 142.03s/it]

best model updated
Epoch 1/150, Train Loss: 0.9539,           Validation Loss: 0.9183, Duration: 0:02:22.023892, Best Val Epoch: 0


  1%|▏         | 2/150 [04:39<5:43:08, 139.11s/it]

best model updated
Epoch 2/150, Train Loss: 0.9443,           Validation Loss: 0.9180, Duration: 0:02:17.070747, Best Val Epoch: 1


  2%|▏         | 3/150 [06:41<5:21:34, 131.25s/it]

...

Epoch 8/150, Train Loss: 0.8650,           Validation Loss: 0.9453, Duration: 0:02:06.037049, Best Val Epoch: 5


  6%|▌         | 9/150 [19:42<5:06:21, 130.37s/it]

best model updated
Epoch 9/150, Train Loss: 0.8520,           Validation Loss: 0.8717, Duration: 0:02:08.729100, Best Val Epoch: 8


  6%|▌         | 9/150 [19:58<5:12:49, 133.12s/it]


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.plot(train_losses, label='train loss')
plt.plot(val_losses, label='validation loss')
plt.title('Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.title('Accuracies')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [6]:
report_k10 = evaluate(model, test_loader, config)

KeyboardInterrupt: 